# Data Analysis





### Import libraries

In [ ]:
import pandas as pd


### Read data

In [ ]:
pd.read_csv('../creditcard.csv')

### General stats about the data

### Clean and wrangle

### Do some useful stuff